In [1]:
import pandas as pd

input_ = pd.read_excel('./5.vehicle_1.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_

,No,Item Number,Market,Page,JOIN_INP,Row,Make,Model,Year,Variant,Type,Chassis,Engine,Notes
0,1,325960692572,UK,1,325960692572;1,1,Mazda,CX-5,2017,Diesel SUV,2.2 D 4WD,"AWD -- GH,KE",2188cc 129KW 175HP SH-VPTS,
1,1,325960692572,UK,1,325960692572;1,2,Mazda,CX-5,2017,Diesel SUV,2.2 D AWD,"AWD -- GH,KE",2191cc 110KW 150HP SHY1,
2,1,325960692572,UK,1,325960692572;1,3,Mazda,CX-5,2017,Diesel SUV,2.2 D AWD,"AWD -- GH,KE",2191cc 129KW 175HP SHY4,
3,1,325960692572,UK,1,325960692572;1,4,Mazda,CX-5,2017,Diesel SUV,2.2 D,"FWD -- GH,KE",2188cc 129KW 175HP SH-VPTS,
4,1,325960692572,UK,1,325960692572;1,5,Mazda,CX-5,2017,Diesel SUV,2.2 D,"FWD -- GH,KE",2191cc 110KW 150HP SHY1;SHY4,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114616,461,224030762701,UK,34,224030762701;34,9,Ford,Transit,1958,Petrol Van,1250 S-2 Klein-LKW,RWD -- --,1487cc 40KW 54HP 15LA,
114617,461,224030762701,UK,34,224030762701;34,10,Ford,Transit,1957,Petrol Bus,K-40 1.5,RWD -- --,1487cc 40KW 54HP 15LA,
114618,461,224030762701,UK,34,224030762701;34,11,Ford,Transit,1957,Petrol Van,1250 S-2 Klein-LKW,RWD -- --,1487cc 40KW 54HP 15LA,
114619,461,224030762701,UK,34,224030762701;34,12,Ford,Transit,1956,Petrol Bus,K-40 1.5,RWD -- --,1487cc 40KW 54HP 15LA,


In [2]:
input_2 = input_[['Item Number']].drop_duplicates(keep='first', ignore_index=True)
input_2 = input_2.sort_values(by=['Item Number'],
                              ascending=[True],
                              ignore_index=True)

input_2

,Item Number
0,223945154954
1,223945185500
2,223945197352
3,223945212805
4,223945212826
...,...
443,326091670945
444,326091671068
445,326091671069
446,326183918987


In [3]:
from tqdm import tqdm

for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
    df_temp = input_[input_['Item Number'] == input_2.loc[i, 'Item Number']].copy().reset_index(drop=True)

    # = = = = = = = = = = = = = = =

    dict_vehicle = {}
    for j in range(len(df_temp)):
        if df_temp.loc[j, 'Market'] in ['US', 'UK', 'AU']:
            make, model, engine, year, year_2 = df_temp.loc[j, 'Make'], df_temp.loc[j, 'Model'], df_temp.loc[j, 'Engine'], int(df_temp.loc[j, 'Year']), int(df_temp.loc[j, 'Year'])
        elif df_temp.loc[j, 'Market'] == 'DE':
            make, model, engine, year, year_2 = df_temp.loc[j, 'Marke'], df_temp.loc[j, 'Modell'], df_temp.loc[j, 'Motor'], int(df_temp.loc[j, 'Baujahr'].split('-')[0].split('/')[0]), int(df_temp.loc[j, 'Baujahr'].split('-')[1].split('/')[0])

        if make not in dict_vehicle.keys():
            dict_vehicle[make] = {model: {engine: [year] if year == year_2 else [year, year_2]}}
        elif model not in dict_vehicle[make].keys():
            dict_vehicle[make][model] = {engine: [year] if year == year_2 else [year, year_2]}
        elif engine not in dict_vehicle[make][model].keys():
            dict_vehicle[make][model][engine] = [year] if year == year_2 else [year, year_2]
        else:
            if year not in dict_vehicle[make][model][engine]:
                dict_vehicle[make][model][engine].append(year)
            if year_2 not in dict_vehicle[make][model][engine]:
                dict_vehicle[make][model][engine].append(year_2)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle.keys():
        for model in dict_vehicle[make].keys():
            list_year = []
            for engine in dict_vehicle[make][model].keys():
                list_year = list(set(list_year + dict_vehicle[make][model][engine]))
            year_begin = min(list_year)
            year_end = max(list_year)
            list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

    input_2.loc[i, 'Vehicle W/o Engine'] = '\n'.join(sorted(list_vehicle))

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle.keys():
        for model in dict_vehicle[make].keys():
            for engine in dict_vehicle[make][model].keys():
                year_begin = min(dict_vehicle[make][model][engine])
                year_end = max(dict_vehicle[make][model][engine])
                list_vehicle.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

    input_2.loc[i, 'Vehicle W/ Engine'] = '\n'.join(sorted(list_vehicle))

# = = = = = = = = = = = = = = =

input_2.to_excel('./6.vehicle.xlsx', index=False)

input_2

Progress: 100%|████████████████████████████| 448/448 [00:05<00:00, 76.57it/s]


,Item Number,Vehicle W/o Engine,Vehicle W/ Engine
0,223945154954,Ford Fiesta 2005-2009,Ford Fiesta 2005-2008 1242cc 51KW 70HP M7JA;M7...
1,223945185500,Toyota Yaris/Vitz 2005-2011,Toyota Yaris/Vitz 2005-2010 1298cc 64KW 87HP 2...
2,223945197352,Ford Focus 2010-2023,Ford Focus 2010-2023 1560cc 70KW 95HP T3DA;T3D...
3,223945212805,Citroën Berlingo 2008-2012\nCitroën Berlingo M...,Citroën Berlingo 2008-2012 1560cc 66KW 90HP 9H...
4,223945212826,Peugeot Partner 2008-2012\nPeugeot Partner Tep...,Peugeot Partner 2008-2012 1560cc 55KW 75HP 9HT...
...,...,...,...
443,326091670945,VW Passat CC 2008-2012,VW Passat CC 2008-2009 1968cc 120KW 163HP CBBA...
444,326091671068,VW Passat CC 2011-2016,VW Passat CC 2011-2013 1968cc 125KW 170HP CFGB...
445,326091671069,VW Passat 2010-2015,VW Passat 2010-2013 1968cc 125KW 170HP CFGB\nV...
446,326183918987,Nissan NV200 / Evalia 2010-2024\nNissan NV200 ...,Nissan NV200 / Evalia 2010-2024 1461cc 63KW 86...
